In [1]:
#Veri İşlemleri
import pandas as pd
import numpy as np
#Veri Kazıma
from bs4 import BeautifulSoup as bs
import requests 
import datetime
from IPython.display import clear_output

In [2]:
sections = ["https://www.hurriyet.com.tr/son-dakika-haberleri/gundem/?p=",
            "https://www.hurriyet.com.tr/son-dakika-haberleri/dunya/?p=",
            "https://www.hurriyet.com.tr/son-dakika-haberleri/ekonomi/?p=",
            "https://www.hurriyet.com.tr/son-dakika-haberleri/sporarena/?p="
           ]

In [8]:
urls = []
for section in sections:
    for i in range(1,11):
        new_url = section + str(i)
        print(new_url)
        html = requests.get(new_url).text
        soup = bs(html, "lxml")
        tags = soup.find("section", class_="son-dakika-contain clr")
        for a in tags.find_all('a', href=True):
            urls.append((section.split("/")[4],a['href']))

https://www.hurriyet.com.tr/son-dakika-haberleri/gundem/?p=1
https://www.hurriyet.com.tr/son-dakika-haberleri/gundem/?p=2
https://www.hurriyet.com.tr/son-dakika-haberleri/gundem/?p=3
https://www.hurriyet.com.tr/son-dakika-haberleri/gundem/?p=4
https://www.hurriyet.com.tr/son-dakika-haberleri/gundem/?p=5
https://www.hurriyet.com.tr/son-dakika-haberleri/gundem/?p=6
https://www.hurriyet.com.tr/son-dakika-haberleri/gundem/?p=7
https://www.hurriyet.com.tr/son-dakika-haberleri/gundem/?p=8
https://www.hurriyet.com.tr/son-dakika-haberleri/gundem/?p=9
https://www.hurriyet.com.tr/son-dakika-haberleri/gundem/?p=10
https://www.hurriyet.com.tr/son-dakika-haberleri/dunya/?p=1
https://www.hurriyet.com.tr/son-dakika-haberleri/dunya/?p=2
https://www.hurriyet.com.tr/son-dakika-haberleri/dunya/?p=3
https://www.hurriyet.com.tr/son-dakika-haberleri/dunya/?p=4
https://www.hurriyet.com.tr/son-dakika-haberleri/dunya/?p=5
https://www.hurriyet.com.tr/son-dakika-haberleri/dunya/?p=6
https://www.hurriyet.com.tr/s

In [9]:
urldata = pd.DataFrame(urls)
urldata.columns = ["Kategori","Link"]
urldata

,Kategori,Link
0,gundem,/gundem/samsunda-yaklasik-90-yabanci-uyruklu-o...
1,gundem,/gundem/tokatta-korkunc-kaza-freni-bosalan-tir...
2,gundem,/gundem/bayram-tatilinde-210-bin-polis-ve-jand...
3,gundem,/gundem/konyada-faciadan-donuldu-kozmetik-fabr...
4,gundem,/gundem/son-dakika-bakan-koca-ilk-kez-diyerek-...
...,...,...
985,sporarena,/sporarena/fenerbahce-aboubakari-almali-torren...
986,sporarena,/sporarena/son-dakika-manchester-united-erik-t...
987,sporarena,/sporarena/evren-turhan-eskiden-para-yok-kalit...
988,sporarena,/sporarena/wimbledonin-rusya-ve-belarus-karari...


In [10]:
urldata = urldata.astype('str')
urldata = urldata.drop_duplicates(subset=None, keep="first", inplace=False)
urldata['Link'] = 'https://www.hurriyet.com.tr' + urldata['Link'].astype(str)

urldata

,Kategori,Link
0,gundem,https://www.hurriyet.com.tr/gundem/samsunda-ya...
1,gundem,https://www.hurriyet.com.tr/gundem/tokatta-kor...
2,gundem,https://www.hurriyet.com.tr/gundem/bayram-tati...
3,gundem,https://www.hurriyet.com.tr/gundem/konyada-fac...
4,gundem,https://www.hurriyet.com.tr/gundem/son-dakika-...
...,...,...
985,sporarena,https://www.hurriyet.com.tr/sporarena/fenerbah...
986,sporarena,https://www.hurriyet.com.tr/sporarena/son-daki...
987,sporarena,https://www.hurriyet.com.tr/sporarena/evren-tu...
988,sporarena,https://www.hurriyet.com.tr/sporarena/wimbledo...


In [11]:
def GetData(url):

    html = requests.get(url).text
    soup = bs(html, "lxml")

    body_text = soup.findAll("div", class_="news-content")[0].findAll('p')

    body_text_big = ""
    for i in body_text:
        body_text_big = body_text_big +i.text

    header = soup.find("h1", class_="news-detail-title").text
    timestamp = soup.find("span", class_="news-date").text

    return ((url,header,body_text_big,timestamp))


In [12]:
bigdata = []
k = 0
for i in urldata.Link:
    clear_output(wait=True)
    print(k)
    bigdata.append(GetData(i))
    k = k + 1

927


In [59]:
bigdatax = pd.DataFrame(bigdata)
bigdatax.drop([4,5,6,7],inplace=True)
bigdatax.columns = ["Haber Linki","Haber Başlığı","Haber İçeriği","Haber Tarihi"]
bigdatax.insert(
      0, 
      "Haber Türü", 
      bigdatax["Haber Linki"].str.split("/",expand=True)[3].str.replace('gundem','Gündem').str.replace('dunya','Dünya').str.replace('ekonomi','Ekonomi').str.replace('sporarena','Spor').str.replace('kelebek','Kelebek-Magazin'))
bigdatax.index = range(0,len(bigdatax))
bigdatax = bigdatax.sample(frac=1).reset_index(drop=True)
bigdatax

,Haber Türü,Haber Linki,Haber Başlığı,Haber İçeriği,Haber Tarihi
0,Ekonomi,https://www.hurriyet.com.tr/ekonomi/pandemide-...,"Pandemide Türkiye, dünyanın odak noktası oldu",İstanbul Demir ve Demir Dışı Metaller İhracatç...,"Nisan 15, 2022 16:09"
1,Ekonomi,https://www.hurriyet.com.tr/ekonomi/asli-taner...,Aslı Taner’in ismi bir okulda yaşatılacak,"Çevre, Şehircilik ve İklim Değişikliği Bakanı ...","Nisan 21, 2022 14:51"
2,Gündem,https://www.hurriyet.com.tr/gundem/bartholomeo...,Bartholomeos: Türkiye'ye gelenler büyük bir mi...,Kıyık semtindeki Sveti Georgi Bulgar Kilisesi'...,"Nisan 30, 2022 16:01"
3,Ekonomi,https://www.hurriyet.com.tr/ekonomi/mb-baskani...,MB Başkanı Kavcıoğlu: Halkımız bize güvensin,"Merkez Bankası Başkanı Şahap Kavcıoğlu, enflas...","Nisan 29, 2022 07:12"
4,Spor,https://www.hurriyet.com.tr/sporarena/fenerbah...,Fenerbahçe'de Serdar Dursun'dan tarihi hat-trick,"Sarı lacivertliler, ligde kalma mücadelesi ver...","Nisan 22, 2022 22:50"
...,...,...,...,...,...
919,Dünya,https://www.hurriyet.com.tr/dunya/disisleri-ba...,"Dışişleri Bakanı Çavuşoğlu, Uruguay'da","Dışişleri Bakanı Mevlüt Çavuşoğlu, Latin Ameri...","Nisan 23, 2022 11:37"
920,Dünya,https://www.hurriyet.com.tr/dunya/putin-ve-zel...,Putin ve Zelenski G20 zirvesinde karşı karşıya...,Dünyanın en büyük ekonomileri arasında yer ala...,"Nisan 29, 2022 17:38"
921,Dünya,https://www.hurriyet.com.tr/dunya/rusyanin-dun...,Rusya’nın Dünya Turizm Örgütü üyeliği askıya a...,Rusya’nın Ukrayna’ya yönelik saldırıları devam...,"Nisan 26, 2022 22:04"
922,Dünya,https://www.hurriyet.com.tr/dunya/ukrayna-bass...,Ukrayna Başsavcılık makamı: Buça'da savaş suçu...,Ukrayna Başsavcısı Iryna Venediktova yaptığı a...,"Nisan 28, 2022 17:12"


In [60]:
bigdatax.to_csv('HurriyetSonDakikaHaberler.csv')

In [61]:
bigdatax.to_excel('HurriyetSonDakikaHaberler.xlsx')

In [62]:
df = pd.read_csv("HurriyetSonDakikaHaberler.csv")
df

,Unnamed: 0,Haber Türü,Haber Linki,Haber Başlığı,Haber İçeriği,Haber Tarihi
0,0,Ekonomi,https://www.hurriyet.com.tr/ekonomi/pandemide-...,"Pandemide Türkiye, dünyanın odak noktası oldu",İstanbul Demir ve Demir Dışı Metaller İhracatç...,"Nisan 15, 2022 16:09"
1,1,Ekonomi,https://www.hurriyet.com.tr/ekonomi/asli-taner...,Aslı Taner’in ismi bir okulda yaşatılacak,"Çevre, Şehircilik ve İklim Değişikliği Bakanı ...","Nisan 21, 2022 14:51"
2,2,Gündem,https://www.hurriyet.com.tr/gundem/bartholomeo...,Bartholomeos: Türkiye'ye gelenler büyük bir mi...,Kıyık semtindeki Sveti Georgi Bulgar Kilisesi'...,"Nisan 30, 2022 16:01"
3,3,Ekonomi,https://www.hurriyet.com.tr/ekonomi/mb-baskani...,MB Başkanı Kavcıoğlu: Halkımız bize güvensin,"Merkez Bankası Başkanı Şahap Kavcıoğlu, enflas...","Nisan 29, 2022 07:12"
4,4,Spor,https://www.hurriyet.com.tr/sporarena/fenerbah...,Fenerbahçe'de Serdar Dursun'dan tarihi hat-trick,"Sarı lacivertliler, ligde kalma mücadelesi ver...","Nisan 22, 2022 22:50"
...,...,...,...,...,...,...
919,919,Dünya,https://www.hurriyet.com.tr/dunya/disisleri-ba...,"Dışişleri Bakanı Çavuşoğlu, Uruguay'da","Dışişleri Bakanı Mevlüt Çavuşoğlu, Latin Ameri...","Nisan 23, 2022 11:37"
920,920,Dünya,https://www.hurriyet.com.tr/dunya/putin-ve-zel...,Putin ve Zelenski G20 zirvesinde karşı karşıya...,Dünyanın en büyük ekonomileri arasında yer ala...,"Nisan 29, 2022 17:38"
921,921,Dünya,https://www.hurriyet.com.tr/dunya/rusyanin-dun...,Rusya’nın Dünya Turizm Örgütü üyeliği askıya a...,Rusya’nın Ukrayna’ya yönelik saldırıları devam...,"Nisan 26, 2022 22:04"
922,922,Dünya,https://www.hurriyet.com.tr/dunya/ukrayna-bass...,Ukrayna Başsavcılık makamı: Buça'da savaş suçu...,Ukrayna Başsavcısı Iryna Venediktova yaptığı a...,"Nisan 28, 2022 17:12"


In [156]:
dff = pd.read_excel("HurriyetSonDakikaHaberler.xlsx")
dff

,Unnamed: 0,Haber Türü,Haber Linki,Haber Başlığı,Haber İçeriği,Haber Tarihi
0,0,Ekonomi,https://www.hurriyet.com.tr/ekonomi/pandemide-...,"Pandemide Türkiye, dünyanın odak noktası oldu",İstanbul Demir ve Demir Dışı Metaller İhracatç...,"Nisan 15, 2022 16:09"
1,1,Ekonomi,https://www.hurriyet.com.tr/ekonomi/asli-taner...,Aslı Taner’in ismi bir okulda yaşatılacak,"Çevre, Şehircilik ve İklim Değişikliği Bakanı ...","Nisan 21, 2022 14:51"
2,2,Gündem,https://www.hurriyet.com.tr/gundem/bartholomeo...,Bartholomeos: Türkiye'ye gelenler büyük bir mi...,Kıyık semtindeki Sveti Georgi Bulgar Kilisesi'...,"Nisan 30, 2022 16:01"
3,3,Ekonomi,https://www.hurriyet.com.tr/ekonomi/mb-baskani...,MB Başkanı Kavcıoğlu: Halkımız bize güvensin,"Merkez Bankası Başkanı Şahap Kavcıoğlu, enflas...","Nisan 29, 2022 07:12"
4,4,Spor,https://www.hurriyet.com.tr/sporarena/fenerbah...,Fenerbahçe'de Serdar Dursun'dan tarihi hat-trick,"Sarı lacivertliler, ligde kalma mücadelesi ver...","Nisan 22, 2022 22:50"
...,...,...,...,...,...,...
919,919,Dünya,https://www.hurriyet.com.tr/dunya/disisleri-ba...,"Dışişleri Bakanı Çavuşoğlu, Uruguay'da","Dışişleri Bakanı Mevlüt Çavuşoğlu, Latin Ameri...","Nisan 23, 2022 11:37"
920,920,Dünya,https://www.hurriyet.com.tr/dunya/putin-ve-zel...,Putin ve Zelenski G20 zirvesinde karşı karşıya...,Dünyanın en büyük ekonomileri arasında yer ala...,"Nisan 29, 2022 17:38"
921,921,Dünya,https://www.hurriyet.com.tr/dunya/rusyanin-dun...,Rusya’nın Dünya Turizm Örgütü üyeliği askıya a...,Rusya’nın Ukrayna’ya yönelik saldırıları devam...,"Nisan 26, 2022 22:04"
922,922,Dünya,https://www.hurriyet.com.tr/dunya/ukrayna-bass...,Ukrayna Başsavcılık makamı: Buça'da savaş suçu...,Ukrayna Başsavcısı Iryna Venediktova yaptığı a...,"Nisan 28, 2022 17:12"


-----------------------------------------Ekstra şeyler...-----------------------------------------

In [169]:
X = df["Haber Başlığı"]
y = df["Haber Türü"]
X = np.array(X)
y = np.array(y)

In [170]:
df["Haber Başlığı"][0]

'Pandemide Türkiye, dünyanın odak noktası oldu'

In [171]:
print("Gündem:",round(len(df[df['Haber Türü']=='Gündem'])/len(df['Haber Türü']),2)*100,"%")
print("Dünya:",round(len(df[df['Haber Türü']=='Dünya'])/len(df['Haber Türü']),2)*100,"%")
print("Ekonomi:",round(len(df[df['Haber Türü']=='Ekonomi'])/len(df['Haber Türü']),2)*100,"%")
print("Spor:",round(len(df[df['Haber Türü']=='Spor'])/len(df['Haber Türü']),2)*100,"%")

Gündem: 8.0 %
Dünya: 31.0 %
Ekonomi: 30.0 %
Spor: 32.0 %


In [172]:
df_gundem = df[df['Haber Türü']=='Gündem']
df_dunya = df[df['Haber Türü']=='Dünya']
df_ekonomi = df[df['Haber Türü']=='Ekonomi']
df_spor = df[df['Haber Türü']=='Spor']

print("Gündem:", df_gundem.shape)
print("Dünya:", df_dunya.shape)
print("Ekonomi:", df_ekonomi.shape)
print("Spor:", df_spor.shape)

Gündem: (75, 6)
Dünya: (282, 6)
Ekonomi: (275, 6)
Spor: (292, 6)


In [173]:
from sklearn.feature_extraction.text import TfidfVectorizer # CountVectorizer and TfiedTransformer birlikte kullanılması
from sklearn.pipeline import Pipeline

from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.preprocessing import LabelEncoder

In [175]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

classifier = Pipeline([
    ('tfidf',TfidfVectorizer(stop_words='english',min_df=1,max_df=0.2)),
    ("method", SVC(kernel = 'sigmoid', degree=20, probability = True))
])
classifier.fit(np.array(X_train), np.array(y_train))

predictions = classifier.predict(X_test)
print(f"{accuracy_score(y_test, predictions)}")
print(classification_report(y_test, predictions))

0.8172043010752689
              precision    recall  f1-score   support

       Dünya       0.81      0.88      0.84        24
     Ekonomi       0.70      0.85      0.77        27
      Gündem       1.00      0.22      0.36         9
        Spor       0.94      0.91      0.92        33

    accuracy                           0.82        93
   macro avg       0.86      0.71      0.72        93
weighted avg       0.84      0.82      0.80        93



In [195]:
print('Haber Türü: ', classifier.predict(["ABD'nin bugün Rusya'dan petrol ihracatını yasaklaması bekleniyor"]))

Haber Türü:  ['Dünya']


In [196]:
print('Haber Türü: ', classifier.predict(["Dev maç öncesi nefesler tutuldu! Galatasaray'ın Barcelona kadrosu belli oldu"]))

Haber Türü:  ['Spor']


In [197]:
print('Haber Türü: ', classifier.predict(["Demir'den önemli tedarik zinciri açıklaması"]))

Haber Türü:  ['Ekonomi']
